In [13]:
import openai
import os   
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import SimpleSequentialChain

load_dotenv(find_dotenv())


openai.api_key = os.getenv('OPENAI_API_KEY')

llm = OpenAI(temperature=1.2)

In [15]:
interpret_prompt = """
    Given a discord message interpret the message:
    
    Tone: the mood of the message? use up to 10 words to describe the tone
    subject: what is the subject of the message? use up to 10 words to describe the subject
    Intent: what is the intent of the message? use up to 10 words to describe the intent
    time: time of message
    Disrespect: on a scale of 1-10 how disrespectful is the message?
    Author: who is the author of the message?
    Message: what is the message?
    relationShip: how do you think the relationship between the author and the recipient is?    
    length: how long of a response is the author looking for? (short, medium, long) estimate if you are not sure


    format the output as JSON with the following keys:
    tone
    subject
    intent
    time
    aggression
    author
    message
    relationship
    length
    



    Message = {input}
 
 """

prompt_interpret = PromptTemplate.from_template(template=interpret_prompt)
chain_interpret = LLMChain(llm=llm, prompt=prompt_interpret, output_key="interpretation")


feelings_prompt = """
    Scenario: You are a bro in a discord server and you are talking to your bros. its where we have our locker room talk, shoot the shit,
    and talk about our pulls of the week. the bros and you are also big into sportsbetting some may even be addicted.
    The group is also into getting hammered and partying. Even though yall are bros sometimes you guys can be harsh to eachother and huge fights break out.

    Character: A total bro that  can be a real asshole. You are extremely confrontational and cant always take a joke. 
    Far-Right Politically and never shys away from a good debate.You are known to using slang sometimes. You have extremely constrovesial takes and never shy away from letting your voice be heard no matter the topic.
    You respond like an asshole sometimes,  gibberish (because your drunk), deep introspective and long(because your high). You are addicted to sports betting and not very good even tho you act like it.
    A total wild card but at the same time the total bro.

    you are given the message, subject, intent, aggression level, author, your relationship with the author
    interpretation = {interpretation}

    given the scenario, character, and interpretation you are tasked with writing a response.


    """ 
prompt_feelings = PromptTemplate.from_template(input_var="interpretation", template=feelings_prompt)
chain_feelings = LLMChain(llm=llm, prompt=prompt_feelings)




overall_chain = SimpleSequentialChain(chains=[chain_interpret, chain_feelings], verbose=True)

response = overall_chain.run(input="we just got running a train on your mom")
print(response)



> Entering new  chain...

Output:
 {
  "tone": "derogatory, mocking, cruel",
  "subject": "sexual content, boastful joking",
  "intent": "humiliate, insult, degrade",
  "time": "<time stamp of message>",
  "aggression": 10,
  "author": "<author of message>",
  "message": "we just got running a train on your mom",
  "relationship": "hostile, strained, acrimonious",
  "length": "none"
 }

Response:
"Hey, ****! That sounds like a really funny joke! But it's probably not the best idea to talk about my mom like that. Don't be a dick, bro."

> Finished chain.

Response:
"Hey, ****! That sounds like a really funny joke! But it's probably not the best idea to talk about my mom like that. Don't be a dick, bro."
